
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/23_Hooks_Anatomy.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🥋 Lekcja 23: Hooks Anatomy (Włamywanie się do modelu)

PyTorch pozwala "wpiąć" własną funkcję w środek działającej sieci bez zmieniania jej klasy. To się nazywa **Hook**.

Mamy dwa główne rodzaje:
1.  **Forward Hook:** Odpala się po obliczeniu wyniku warstwy.
    *   *Użycie:* Debugowanie kształtów, wyciąganie cech (Feature Extraction), wizualizacja aktywacji.
2.  **Backward Hook:** Odpala się podczas liczenia gradientów.
    *   *Użycie:* Debugowanie znikających gradientów (NaN), modyfikacja treningu (Gradient Clipping).

**Zasada:** Hook to funkcja, którą rejestrujesz. Zwraca ona `handle` (uchwyt), którego musisz użyć, żeby ją potem usunąć (`handle.remove()`). Inaczej zostanie tam na zawsze!

In [1]:
import torch
import torch.nn as nn

# Prosty model
model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 2)
)

print("Model gotowy. Na razie to czarna skrzynka.")

Model gotowy. Na razie to czarna skrzynka.


## 1. Forward Hook (Szpieg)

Chcemy zobaczyć, jakie liczby wychodzą z warstwy `ReLU` (środkowej), ale funkcja `model(x)` zwraca tylko wynik końcowy.

Napiszemy hooka, który wypisze kształt i średnią wartość aktywacji.
Sygnatura funkcji: `hook(module, input, output)`.

In [2]:
# Nasz szpieg
def print_activations(module, input, output):
    print(f"\n🕵️ HOOK na warstwie: {module}")
    print(f"   Input shape:  {input[0].shape}")
    print(f"   Output shape: {output.shape}")
    print(f"   Średnia aktywacja: {output.mean().item():.4f}")

# Wybieramy warstwę, którą chcemy podsłuchać (indeks 1 to ReLU)
target_layer = model[1]

# Rejestracja (Zapisujemy uchwyt, żeby móc to wyłączyć!)
handle = target_layer.register_forward_hook(print_activations)

print("Hook zarejestrowany na ReLU.")

# Uruchamiamy model
x = torch.randn(1, 10)
out = model(x)

print("\n--- Koniec Forward Pass ---")

Hook zarejestrowany na ReLU.

🕵️ HOOK na warstwie: ReLU()
   Input shape:  torch.Size([1, 5])
   Output shape: torch.Size([1, 5])
   Średnia aktywacja: 0.2161

--- Koniec Forward Pass ---


## Sprzątanie (Bardzo ważne!)

Hooki są globalne dla obiektu. Jeśli uruchomisz model 100 razy, hook odpali się 100 razy.
Musisz je usuwać po użyciu.

In [3]:
# Usuwamy szpiega
handle.remove()
print("Hook usunięty.")

# Sprawdzenie (cisza w eterze)
out = model(x)
print("Model działa po cichu.")

Hook usunięty.
Model działa po cichu.


## 2. Backward Hook (Haker)

Backward hook pozwala nie tylko "patrzeć", ale też **zmieniać** gradienty.
Możemy np. zamrozić warstwę (wyzerować gradient) albo zrobić Gradient Clipping, nie dotykając pętli treningowej.

Sygnatura: `hook(module, grad_input, grad_output)`.
Jeśli funkcja zwróci nowy tensor, zastąpi on oryginalny gradient!

In [4]:
# Funkcja hakująca gradient (Wersja bezpieczna)
def cap_gradients(module, grad_input, grad_output):
    print(f"\n🔧 BACKWARD HOOK na: {module}")
    
    # Tworzymy nową listę gradientów wejściowych
    new_grad_input = []
    
    # Iterujemy po wszystkim, co wchodzi (Input, Weights, Bias)
    for i, g in enumerate(grad_input):
        if g is not None:
            print(f"   Zeruję element {i} (shape: {g.shape})")
            new_grad_input.append(torch.zeros_like(g))
        else:
            new_grad_input.append(None)
            
    # Musimy zwrócić KROTKĘ (Tuple)
    return tuple(new_grad_input)

# Rejestrujemy na ostatniej warstwie (Linear)
layer = model[2]
handle_back = layer.register_full_backward_hook(cap_gradients)

# Forward
out = model(x)
loss = out.sum()

# Backward (Tu odpali się hook)
print("--- Start Backward ---")
loss.backward()

print(f"\nSprawdźmy gradienty wag PIERWSZEJ warstwy:")
# Powinny być zerowe.
# Dlaczego? Bo ostatnia warstwa (gdzie jest hook) wysłała w dół same zera.
# Zera pomnożone przez cokolwiek dają zera. Sygnał uczenia został odcięty.
print(model[0].weight.grad) 

# Sprzątanie
handle_back.remove()

--- Start Backward ---

🔧 BACKWARD HOOK na: Linear(in_features=5, out_features=2, bias=True)
   Zeruję element 0 (shape: torch.Size([1, 5]))

Sprawdźmy gradienty wag PIERWSZEJ warstwy:
tensor([[-0., 0., -0., 0., 0., -0., 0., -0., 0., 0.],
        [-0., 0., -0., 0., 0., -0., 0., -0., 0., 0.],
        [-0., 0., -0., 0., 0., -0., 0., -0., 0., 0.],
        [-0., 0., -0., 0., 0., -0., 0., -0., 0., 0.],
        [-0., 0., -0., 0., 0., -0., 0., -0., 0., 0.]])


## 🥋 Black Belt Summary

1.  **Forward Hooks:** Używane w **Neural Style Transfer** (żeby pobrać styl z warstw środkowych) i w **Feature Extraction** (np. gdy używasz ResNet jako backbone do detekcji obiektów).
2.  **Backward Hooks:** Używane do debugowania **Vanishing Gradients** (widzisz, gdzie gradient zmienia się w zero) lub do zaawansowanych modyfikacji treningu.
3.  **Pamięć:** Jeśli w hooku zapisujesz tensory na liście (np. `self.activations.append(output)`), pamiętaj, żeby je czyścić! Inaczej zapchasz VRAM w kilka minut, bo PyTorch będzie trzymał historię grafu dla każdego zapisanego tensora.